# settings

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
import geopandas as gpd
from sklearn.neighbors import BallTree
from sklearn.linear_model import LinearRegression


In [2]:
# importing all data
#frame = pd.read_pickle('Data\ANA_clean.pkl')
#gdf = gpd.read_file('Data\stations.geojson')
frame = pd.read_pickle(r'Data\all_clean.pkl')
gdf = gpd.read_file(r'Data\all_stations.geojson')

In [3]:
# define the thresholds
start = '1981-01-01'
end = '2010-12-31'
na_lim = 20
std_lim = 2.7
between_limit = 0.75
file_name = 'Data\ALL_81_10_20bet'

# Filters

In [4]:
def filter_df(start, end, na, frame = frame):
    """
    Filter the DataFrame by date and remove stations with too many missing values.

    Parameters:
        start (datetime): The starting date of the desired date range (inclusive).
        end (datetime): The ending date of the desired date range (inclusive).
        na (float): The threshold percentage of missing values allowed for a station to be included in the filtered DataFrame.
        frame (pandas.DataFrame, optional): The input DataFrame to be filtered. 

    Returns:
        df_final (pandas.DataFrame): The filtered DataFrame containing the stations that meet the date range criteria and have a percentage of missing values below the threshold.

    """
  
    date_begin = start
    date_end = end
    #selectt data in frame in start-end period
    frame_dates = frame[(frame['Date'] >= start) & (frame['Date'] <= end)]
    date_range = pd.date_range(start = date_begin, end = date_end, freq='MS' )
    codes = frame_dates['Code'].unique()
    df_na = frame_dates.dropna()
    diff_list = []
    for code in codes:
        diff = date_range.difference(df_na[df_na['Code']==code]['Date'])
        diff_list.append(len(diff))
    difs_date_series = pd.Series(diff_list, index=codes)
    values_na = difs_date_series / len(date_range) *100
    na_codes_list= values_na[values_na < na].index.to_list()
    df_final = df_na[df_na['Code'].isin(na_codes_list)]

    return df_final

In [5]:
df_dates = filter_df(start, end, na_lim)

In [6]:
def filter_raw(df, gdf = gdf):
    """
    Get only raw data from the filtered DataFrame and drop duplicated stations and stations with dates out of the pattern (first day of the month).

    Parameters:
        df (pandas.DataFrame): The filtered DataFrame containing the stations and dates.
        gdf (pandas.DataFrame, optional): The geographical DataFrame containing station information. If not provided, the function assumes a global variable named "gdf" exists.

    Returns:
        df_raw (pandas.DataFrame): The DataFrame containing only raw data without duplicated stations and dates out of the pattern.
        gdf_filtered (pandas.DataFrame): The geographical DataFrame filtered to include only the stations present in df_raw.

    """
    codes_out = df[df['Date'].dt.day != 1]['Code'].unique()
    df = df[~df.Code.isin(codes_out)]
    df_con = df.sort_values(by='Consistency').drop_duplicates(subset=['Code','Date'], keep='first')
    df_raw = df_con.set_index(['Code','Date']).unstack().stack(dropna=False).reset_index()
    codes_out = df_raw[df_raw['Date'].dt.day != 1]['Code'].unique()
    df_raw = df_raw[~df_raw.Code.isin(codes_out)]
    gdf_filtered = gdf[gdf['Code'].isin(df_raw['Code'].unique())]
    gdf_filtered = gdf_filtered.sort_values(by='Code').drop_duplicates(subset=['Latitude','Longitude'], keep='first')
    df_raw = df_raw[df_raw['Code'].isin(gdf_filtered['Code'].unique())]
    return df_raw , gdf_filtered

In [7]:
df_raw, gdf_filtered = filter_raw(df_dates)

In [8]:
stat_wrong_dates = df_dates[df_dates['Date'].dt.day != 1]['Code'].unique().shape[0]
stat_dup = df_dates.Code.unique().shape[0] - df_raw.Code.unique().shape[0]  - stat_wrong_dates

In [9]:
gdf_filtered.reset_index(inplace=True, drop=True)
# using a projection to get the distance in meters
gdf_filtered_dist = gdf_filtered.to_crs('ESRI:102032')

In [10]:
def get_neighbors(src_points, candidates, k_neighbors):
    """Find nearest neighbors for all source points from a set of candidate points"""

    # Create tree from the candidate points
    tree = BallTree(candidates, leaf_size=40, metric='haversine')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    return indices, distances


def nearest_neighbors(left_gdf, right_gdf, k_neighbors=6):
    """
    For each point in left_gdf, find closest point in right GeoDataFrame and return them.
    
    NOTICE: Assumes that the input Points are in WGS84 projection (lat/lon).
    """
    
    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name
    
    # Ensure that index in right gdf is formed of sequential numbers
    right = right_gdf.copy().reset_index(drop=True)
    
    # Parse coordinates from points and insert them into a numpy array as RADIANS
    # Notice: should be in Lat/Lon format 
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.y * np.pi / 180, geom.x * np.pi / 180)).to_list())
        
    closest, dist = get_neighbors(src_points=left_radians, candidates=right_radians, k_neighbors = k_neighbors)
        
    return closest

In [11]:
index_near = nearest_neighbors(gdf_filtered.to_crs('EPSG:4326'), gdf_filtered.to_crs('EPSG:4326'), k_neighbors=6)

In [12]:
def detect_outliers(df1, std_lim):
    groups = df1.groupby([df1.Code, df1.Date.dt.month])['Total']
    mean, std = groups.transform("mean"), groups.transform("std")
    normalized = (df1['Total'] - mean) / std
    df1['Normalized'] = normalized
    outliers = df1[(df1['Normalized'] >= std_lim) | (df1['Normalized'] <= - std_lim)]
    return outliers

def outliers_neighbors(row, idx = index_near, std = std_lim):
    code = row.Code
    date = row.Date
    indexx = (gdf_filtered['Code'] == code).idxmax()
    val = gdf_filtered.iloc[idx[:, indexx]]['Code'].values
    out_close = df_raw[(df_raw['Code'].isin(val) )& (df_raw['Date'] == date)]
    return (out_close['Normalized'].abs() >= std).sum()

def drop_outliers(df, std_lim):
    outliers = detect_outliers(df, std_lim)
    results_out = outliers[outliers.apply(outliers_neighbors, axis=1) == 1]
    df_out_drop = df.drop(results_out.index)
    return df_out_drop

In [13]:
df_out_drop = drop_outliers(df_raw, std_lim)

In [14]:
dropped_outliers = (df_raw.shape[0] - df_out_drop.shape[0]) / df_raw.shape[0] * 100

In [15]:
df_out_drop.drop(columns=['Normalized'], inplace=True)
df_out_drop = df_out_drop.set_index(['Code','Date']).unstack().stack(dropna=False).reset_index()


In [16]:
df_na = df_out_drop[df_out_drop['Total'].isna()]

In [17]:
def weighted_mean(df, values, weights, groupby):
    '''
    calculate the weighted mean of a dataframe
    '''
    df = df.copy()
    grouped = df.groupby(groupby)
    df['weighted_average'] = df[values] / grouped[weights].transform('sum') * df[weights]
    return grouped['weighted_average'].sum(min_count=1) #min_count is required for Grouper objects

In [18]:
def fill_na_RV(row, idx = index_near):
    '''
    fill the missing values with the weighted mean of the nearest neighbors
    '''
    code = row.Code
    date = row.Date
    indexx = (gdf_filtered_dist['Code'] == code).idxmax()
    val = gdf_filtered_dist.iloc[idx[:, indexx]]['Code'].values
    # getting the nearest neighbors data
    close = df_out_drop[(df_out_drop['Code'].isin(val)) ]
    close = close[close.Date.dt.month == row.Date.month]
    # calculating the weighted mean
    
    stat = gdf_filtered_dist.iloc[[indexx]]
    stations = gdf_filtered_dist[(gdf_filtered_dist.Code.isin(val)) & (gdf_filtered_dist.Code != code)]
    stations['dist'] = 1/(stations['geometry'].apply(lambda x: stat.distance(x)))

    df_stat = close[close.Code == code]
    close = close[close.Code != code]
    close = close.merge(stations[['Code', 'dist']], on='Code')
    mean_close = weighted_mean(close, 'Total', 'dist', 'Date')
    # fitting the linear_model
    x = mean_close.values
    y = df_stat['Total'].values
    x = x[~np.isnan(y)]
    y = y[~np.isnan(y)]
    y = y[~np.isnan(x)]
    x = x[~np.isnan(x)]
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    target = mean_close.loc[date]
    if np.isnan(target):
        predict = np.nan
    else:    
        model = LinearRegression()
        model.fit(x, y)
        predict = model.predict(target.reshape(-1, 1))[0][0]
    return predict

In [19]:
NA_filled_RV = df_na.apply(fill_na_RV , axis=1)


In [20]:
df_na['RV'] = NA_filled_RV
na_clim = df_na['RV'].isnull().sum()
na_reg = df_na.shape[0] - na_clim

C:\Users\luisg\AppData\Local\Temp\ipykernel_26220\3356163064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_na['RV'] = NA_filled_RV


In [21]:
df_filled = df_out_drop.copy()
df_filled['Total'] = df_out_drop['Total'].fillna(df_na['RV'])
df_filled['Total'] = df_filled.groupby(['Code', df_filled.Date.dt.month], group_keys=False)['Total'].apply(lambda x: x.fillna(x.mean()))

In [22]:
def double_mass_curve(row, idx= index_near):
    '''
    fitting the linear regression model to the nearest neighbors
    '''
    code = row.Code
    date = row.Date
    indexx = (gdf_filtered_dist['Code'] == code).idxmax()
    val = gdf_filtered_dist.iloc[idx[:, indexx]]['Code'].values
    close = df_filled[(df_filled['Code'].isin(val)) ]
    close = close[close.Date.dt.month == row.Date]
    mean_close = close[close.Code != code].groupby(close.Date).Total.mean()
    stat = gdf_filtered_dist.iloc[[indexx]]
    stations = gdf_filtered_dist[(gdf_filtered_dist.Code.isin(val)) & (gdf_filtered_dist.Code != code)]
    stations['dist'] = 1/(stations['geometry'].apply(lambda x: stat.distance(x)))
    df_stat = close[close.Code == code]
    close = close[close.Code != code]
    close = close.merge(stations[['Code', 'dist']], on='Code')
    mean_close = weighted_mean(close, 'Total', 'dist', 'Date')

    x = mean_close.values
    y = df_stat['Total'].values
    x = x.reshape(-1, 1)
    y = y.reshape(-1, 1)
    
    model = LinearRegression()
    model.fit(x, y)
# calculate the line using the model
    yhat = model.predict(x)
    df_stat['predict'] = yhat
    return pd.Series(yhat.squeeze())
#cada mes de cada estação



In [23]:
df_cod_dates = df_filled.groupby(['Code', df_filled.Date.dt.month]).mean().reset_index().drop(columns=['Total'])
consist_values = df_cod_dates.apply(double_mass_curve , axis=1)
df_test = df_filled.copy()
df_test['month'] = df_filled.Date.dt.month
df_test = df_test.sort_values(by=['Code', 'month'])
df_test['predict'] = consist_values.to_numpy().flatten()
df_test.drop(columns=['month'], inplace=True)
df_test['predict'] = df_test['predict'].clip(0)


C:\Users\luisg\AppData\Local\Temp\ipykernel_26220\3503087231.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_cod_dates = df_filled.groupby(['Code', df_filled.Date.dt.month]).mean().reset_index().drop(columns=['Total'])


In [24]:
def between(a,b,x):
    return (x >= a) & (x <= b)


In [25]:
def between_consistency(row, inf=1-between_limit, sup=1 + between_limit):
    value = 0
    if between(row['predict']*inf, row['predict']*sup, row['Total']):
        value = row['Total']
    else:
        value = row['predict']
    return value  

In [26]:
df_between = df_test.copy()
df_between['between'] = df_between.apply(between_consistency, axis=1)

In [27]:
consist_data = (df_test['Total'] != df_between['Total']).sum() / df_between.shape[0] * 100

In [28]:
df = df_between[['Code', 'Date', 'between']].rename(columns={'between': 'Total'})


# results

In [29]:
print('start' , start)
print('end' , end)
print('na_lim', na_lim)
print('std_lim', std_lim)
print(df_dates.Code.unique().shape[0], 'stations after Date and NA Filter')
print(stat_wrong_dates, 'stations with broken dates')
print(stat_dup, 'duplicated stations')
print(f'{dropped_outliers:.2f}% dropped outliers')
print(na_reg, 'NA values filled with linear regression')
print(na_clim, 'NA values filled with climatology')
print(f'{consist_data:.2f}% consisted data')



start 1981-01-01
end 2010-12-31
na_lim 20
std_lim 2.7
4622 stations after Date and NA Filter
2 stations with broken dates
17 duplicated stations
0.65% dropped outliers
66351 NA values filled with linear regression
2026 NA values filled with climatology
0.00% consisted data


In [30]:
df.to_pickle(file_name + '.pkl')

